# Algoritmos de Machine Learning

En este Notebook, usaremos los Dataframes creados en el Notebook de Features para entrenar distintos modelos de Machine Learning y predecir si los tweets son reales o no.

In [1]:
import pandas as pd
import numpy as np

import warnings 
warnings.filterwarnings('ignore')

## Carga de dataframes

In [2]:
train_df = pd.read_csv('../Data/train_features.csv', encoding='latin-1',dtype={'id': np.uint16,'target': np.bool})
test_df = pd.read_csv('../Data/test_features.csv', encoding='latin-1',dtype={'id': np.uint16})

In [3]:
train_df.head(2)

,id,target,cantidad_de_palabras,longitud_del_tweet,tiene_menciones,es_expresivo,cantidad_de_hashtags,tiene_links
0,1,True,13,69,False,False,1,False
1,4,True,7,38,False,False,0,False


In [4]:
test_df.head(2)

,id,cantidad_de_palabras,longitud_del_tweet,tiene_menciones,es_expresivo,cantidad_de_hashtags,tiene_links
0,0,6,34,False,False,0,False
1,2,9,64,False,False,1,False


## Algoritmos

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

### Random Forest

###### Entrenamiento

In [6]:
X = train_df.sort_values(by='id',ascending=True).iloc[:,2:].values
Y = train_df.sort_values(by='id',ascending=True).iloc[:,1].values

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=1)

In [8]:
from sklearn.ensemble import RandomForestClassifier

accuracy = 0
cant_arboles_optimos = 0

for i in range(100):
    classifier = RandomForestClassifier(n_estimators=i+1, random_state=1)
    classifier.fit(X_train, Y_train)
    Y_pred = classifier.predict(X_test)
        
    if (accuracy < accuracy_score(Y_test,Y_pred)):
        accuracy = accuracy_score(Y_test,Y_pred)
        cant_arboles_optimos = i+1

In [9]:
print('Arboles:',cant_arboles_optimos)
print('Accuracy:',accuracy)

Arboles: 18
Accuracy: 0.6657912015758372


In [10]:
classifier = RandomForestClassifier(n_estimators=cant_arboles_optimos, random_state=1)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)

In [11]:
print(classification_report(Y_test,Y_pred))
print(accuracy_score(Y_test,Y_pred))

              precision    recall  f1-score   support

       False       0.70      0.75      0.72       882
        True       0.61      0.55      0.58       641

    accuracy                           0.67      1523
   macro avg       0.66      0.65      0.65      1523
weighted avg       0.66      0.67      0.66      1523

0.6657912015758372


###### Predicción

In [12]:
registros_a_predecir = test_df.sort_values(by='id',ascending=True).iloc[:,1:].values

In [13]:
prediccion = classifier.predict(registros_a_predecir)

Damos el formato para realizar el submit.

In [14]:
resultado_RF = pd.DataFrame(pd.Series(prediccion))
resultado_RF.insert(loc=0, column='id', value=(test_df.sort_values(by='id',ascending=True)['id']))
resultado_RF.rename(columns={0:'target'},inplace=True)
resultado_RF.set_index('id',inplace=True)
resultado_RF['target'] = resultado_RF['target'].astype('uint8')
resultado_RF.head()

,target
id,
0,0
2,0
3,0
9,0
11,0


In [15]:
resultado_RF.to_csv('prediccion_RF.csv')

### Redes Neuronales

###### Entrenamiento

Pasamos las variables a categoricas/booleanas a enteros.

In [16]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

train_df['tiene_menciones_int'] = encoder.fit_transform(train_df.tiene_menciones.values)
train_df['es_expresivo_int'] = encoder.fit_transform(train_df.es_expresivo.values)
train_df['tiene_link_int'] = encoder.fit_transform(train_df.tiene_links.values)
train_df['target_int'] = encoder.fit_transform(train_df.target.values)

test_df['tiene_menciones_int'] = encoder.fit_transform(test_df.tiene_menciones.values)
test_df['es_expresivo_int'] = encoder.fit_transform(test_df.es_expresivo.values)
test_df['tiene_link_int'] = encoder.fit_transform(test_df.tiene_links.values)


In [17]:
train_int = train_df[['cantidad_de_palabras','longitud_del_tweet','tiene_menciones_int','es_expresivo_int','cantidad_de_hashtags','tiene_link_int','target']]
test_int = test_df[['cantidad_de_palabras','longitud_del_tweet','tiene_menciones_int','es_expresivo_int','cantidad_de_hashtags','tiene_link_int']]

#Sin la columna "tiene link" calculó mejor
X = train_int.iloc[:,:-2].values
Y = train_int.iloc[:,-1].values

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=1)

In [23]:
#Normalizar
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(10,10,10), max_iter=500, alpha=0.0001,
                    solver='adam', random_state=5,tol=0.0000001)
mlp.fit(X_train, Y_train)
Y_pred = mlp.predict(X_test)

In [27]:
print(classification_report(Y_test,Y_pred))
print(accuracy_score(Y_test,Y_pred))

              precision    recall  f1-score   support

       False       0.71      0.73      0.72       882
        True       0.61      0.59      0.60       641

    accuracy                           0.67      1523
   macro avg       0.66      0.66      0.66      1523
weighted avg       0.67      0.67      0.67      1523

0.6677609980302035


###### Predicción

In [28]:
registros_a_predecir = test_int.iloc[:,:-1].values
registros_a_predecir = scaler.transform(registros_a_predecir)

In [29]:
prediccion = mlp.predict(registros_a_predecir)

Damos el formato para realizar el submit.

In [30]:
resultado_RN = pd.DataFrame(pd.Series(prediccion))
resultado_RN.insert(loc=0, column='id', value=(test_df.sort_values(by='id',ascending=True)['id']))
resultado_RN.rename(columns={0:'target'},inplace=True)
resultado_RN.set_index('id',inplace=True)
resultado_RN['target'] = resultado_RN['target'].astype('uint8')
resultado_RN.head()

,target
id,
0,0
2,0
3,0
9,0
11,0


In [31]:
resultado_RN.to_csv('prediccion_RN.csv')